# Optimizavimo metodų laboratorinis darbas
#### Elijas Dapšauskas

In [1]:
if (!require(linprog)) install.packages('linprog')
library('linprog')
library('lpSolve')

Loading required package: linprog
Warning message:
"package 'linprog' was built under R version 3.5.3"Loading required package: lpSolve
Warning message:
"package 'lpSolve' was built under R version 3.5.3"

# 1.
### 1.1.
##### 1.1.1.

$x_1$ &mdash; užsakomos minutės radijuje<br>
$x_2$ &mdash; užsakomos minutės televizijoje

$$x_1+25x_2\rightarrow\max$$
$$\begin{cases} 
15x_1 + 300x_2 \leq 10000 \\
-x_1+2x_2 \leq 0 \\
x_1 \leq 400 \\
x_1,x_2 \geq 0
\end{cases}$$

In [5]:
radio.cost_per_min = 15
tv.cost_per_min = 300

optimizeAdBudget = function(available_budget) {
    cvec = c(1,25)
    Amat = rbind(c(radio.cost_per_min, tv.cost_per_min),
                 c(-1, 2),
                 c(1, 0))
    bvec= c(available_budget,
            0,
            400)
    solution = solveLP(cvec,bvec,Amat,maximum=T)$solution
    return(list(radio_mins=solution[1], tv_mins=solution[2]))
}

ad_budget = optimizeAdBudget(available_budget=10000)
print(paste('Televizijoje:', floor(ad_budget$tv_mins), 'min.'))
print(paste('Radijuje:', floor(ad_budget$radio_mins), 'min.'))

[1] "Televizijoje: 30 min."
[1] "Radijuje: 60 min."


##### 1.1.2.

In [3]:
monthly_radio_budget = radio.cost_per_min * floor(ad_budget$radio_mins)
print(paste(monthly_radio_budget, 'EUR'))

[1] "900 EUR"


##### 1.1.3.

In [4]:
ad_budget = optimizeAdBudget(available_budget=15000)
print(paste('Televizijoje:', floor(ad_budget$tv_mins), 'min.'))
print(paste('Radijuje:', floor(ad_budget$radio_mins), 'min.'))

[1] "Televizijoje: 45 min."
[1] "Radijuje: 90 min."


# 2.
### 2.1.
##### 2.1.1.

$x_i$ &mdash; "Ar stotis pastatoma $i$-ajame mieste?"<br>
$x_i \in \{0,1\}$ &mdash; čia $1$ reiškia "Taip" <br>
Apribojimai sudaromi taip, kad būtų bent vienas ($\geq 1$) miestas su pagalbos stotimi, iš kurio galima patekti į $i$-ąjį miestą per ne daugiau 15 min. Pavyzdžiui, į pirmąjį miestą šitaip galime patekti iš 1, 3, 4 miestų, todėl apribojimas $x_1+x_3+x_4 \geq 1$.

$$x_1+x_2+x_3+x_4+x_5+x_6\rightarrow\min$$
$$\begin{cases} 
x_1+x_3+x_4 \geq 1 \\
x_2+x_4+x_6 \geq 1 \\
x_1+x_3 \geq 1 \\
x_2+x_4 \geq 1 \\
x_1+x_5+x_6 \geq 1 \\
x_2+x_5+x_6 \geq 1
\end{cases}$$

##### 2.1.2.

Iš uždavinio sąlygos:

In [14]:
cities = 6
distance_limit = 15
distances = rbind(c(0,23,14,18,10,32),
                  c(23,0,24,13,22,11),
                  c(14,24,0,60,19,20),
                  c(18,13,60,0,55,17),
                  c(10,22,19,55,0,12),
                  c(32,11,20,17,12,0))

Išsprendžiame TPU:

In [16]:
cvec = rep(1, cities)
bvec = rep(1, cities)
Amat = distances <= distance_limit
solution = lp("min",cvec,Amat,'>=',bvec,int.vec=1:length(cvec),all.bin=T)$solution

print(paste("Mažiausias reikalingų stočių kiekis:", sum(solution)))
print(paste(c("Miestai kuriose statomos stotys:", which(solution>0)), collapse=" "))

[1] "Mažiausias reikalingų stočių kiekis: 2"
[1] "Miestai kuriose statomos stotys: 1 2"


### 2.2.
##### 2.2.1.

### 2.3.
##### 2.3.1.

In [9]:
# Visos sumos skaičiuojamos 1 tūkst. eur matu
K = 1e4
profit = c(50, 100, 200, 300, 80, 200)
expenses = c(1000, 2000, 3000, 5000, 1500, 3500)

In [10]:
cvec = profit
Amat = rbind(expenses)
bvec = K

result = lp("max",cvec,Amat,'<=',bvec,int.vec=1:length(cvec),all.bin=T)
print(paste(c("Vykdomi projektai:", which(result$solution>0)), collapse=" "))

[1] "Vykdomi projektai: 2 3 4"


##### 2.3.2.

In [11]:
cvec = profit
Amat = rbind(expenses,
             c(1,0,0,0,0,0),
             c(0,1,0,0,0,0),
             c(0,0,1,0,0,0),
             c(0,0,0,1,0,0),
             c(0,0,0,0,1,0),
             c(0,0,0,0,0,1))
bvec = c(K,1,1,1,1,1,1)

result = lp("max",cvec,Amat,'<=',bvec,compute.sens=T)
print(paste(c("Vykdomi projektai:", which(result$solution>0)), collapse=" "))
print(paste(c("Projektų vykdymo koeficientai:", result$solution), collapse=" "))
print(paste(c("Jautrumo analizė. Nuo:", result$sens.coef.from), collapse=" "))
print(paste(c("Jautrumo analizė. Iki:", result$sens.coef.to), collapse=" "))

[1] "Vykdomi projektai: 3 4 6"
[1] "Projektų vykdymo koeficientai: 0 0 1 1 0 0.571428571428572"
[1] "Jautrumo analizė. Nuo: -1e+30 -1e+30 171.428571428571 285.714285714286 -1e+30 186.666666666667"
[1] "Jautrumo analizė. Iki: 57.1428571428571 114.285714285714 1e+30 1e+30 85.7142857142857 210"


### 2.4.
##### 2.4.1.

In [12]:
door_count = 8
room_count = 8
bvec = rep(1,room_count)
cvec = rep(1,door_count)
Amat = rbind(c(1,1,0,0,0,0,0,0),
             c(1,0,1,0,0,0,0,0),
             c(0,1,0,1,0,0,0,0),
             c(0,0,0,1,1,0,0,0),
             c(0,0,1,0,1,1,0,0),
             c(0,0,0,0,0,0,1,0),
             c(0,0,0,0,0,0,1,1),
             c(0,0,0,0,0,1,0,1))
solution = lp("min",cvec,Amat,'>=',bvec,int.vec=1:length(cvec),all.bin=TRUE)$solution
print(paste("Mažiausias reikalingų sargų kiekis:", sum(solution)))
print(paste(c("Tarpduriai kuriuose statomi sargai:", which(solution>0)), collapse=" "))

[1] "Mažiausias reikalingų sargų kiekis: 4"
[1] "Tarpduriai kuriuose statomi sargai: 1 4 6 7"


### 2.5.

Žymėsime tarpdurius skaičiais nuo 1 iki 8, eilės tvarka iš viršaus į apačią, tada iš kairės į dešinę. $x_i \in \{0,1\}$ žymėsime, ar pastatyti sargą į $i$-ąjį tarpdurį. Apribojimus sudarysime taip, kad bent viename kambariui priklausančiui tarpduryje būtų pastatytas sargas, pavyzdžiui viršutiniam kairiajam kambariui priklauso tarpduriai 1 ir 2, todėl turėsime apribojimą $x_1 + x_2 \geq 1$. Analogiškai sudarysime apribojimus ir likusiems kambariams. Kambariai surašomi tokia tvarka: iš viršaus į apačią, tada - iš kairės į dešinę.

In [12]:
door_count = 8
room_count = 8
bvec = rep(1,room_count)
cvec = rep(1,door_count)
Amat = rbind(c(1,1,0,0,0,0,0,0),
             c(1,0,1,0,0,0,0,0),
             c(0,1,0,1,0,0,0,0),
             c(0,0,0,1,1,0,0,0),
             c(0,0,1,0,1,1,0,0),
             c(0,0,0,0,0,0,1,0),
             c(0,0,0,0,0,0,1,1),
             c(0,0,0,0,0,1,0,1))
solution = lp("min",cvec,Amat,'>=',bvec,int.vec=1:length(cvec),all.bin=TRUE)$solution
print(paste("Mažiausias reikalingų sargų kiekis:", sum(solution)))
print(paste(c("Tarpduriai kuriuose statomi sargai:", which(solution>0)), collapse=" "))

[1] "Mažiausias reikalingų sargų kiekis: 4"
[1] "Tarpduriai kuriuose statomi sargai: 1 4 6 7"


In [13]:
# Final value (z)
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,all.bin=TRUE)

# Variables final values
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,all.bin=TRUE)$solution

# Sensitivities
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,compute.sens=TRUE,all.bin=TRUE)$sens.coef.from
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,compute.sens=TRUE,all.bin=TRUE)$sens.coef.to

# Dual Values (first dual of the constraints and then dual of the variables)
# Duals of the constraints and variables are mixed
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,compute.sens=TRUE,all.bin=TRUE)$duals

# Duals lower and upper limits:
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,compute.sens=TRUE,all.bin=TRUE)$duals.from
lp("max",f.obj,f.con,f.dir,f.rhs,int.vec=1:4,compute.sens=TRUE,all.bin=TRUE)$duals.to

ERROR: Error in is.data.frame(objective.in): object 'f.obj' not found


# set.seed(1029)

n=10
t = c( 1: n)
t

x = rnorm (n, 0, 4)
x


ep = runif (n, -5, 5)
ep
vid = mean(ep)
vid
var = sqrt( var(ep))
var

# sukuriame musu a ir b

a = 2
b = 5

y = a + b*x +ep
y
plot(y)

plot(x,y)
abline(a, b, col = 2)

v = replicate (n, 1)
v

XX = cbind (v, x)
XX

betahat = solve (t(XX) %*% XX) %*% (t(XX) %*% y)
betahat 

ahat = betahat [1]
ahat

bhat = betahat [2]
bhat

lm1 = lm ( y ~ x)
lm1
summary(lm1)

plot(x,y)
abline(a, b, col = 2) # musu sugeneruotos reiksmes a ir b
abline(betahat, col = 3) # kompiuterio sugeneruotos reiksmes ahat ir bhat


# ANTRA DALIS ---------------------------

p = 1/2
z <- rbinom (n, 1, prob = p)
z

z1 = 1-z
z1

XX1 = cbind (z1, z1*x, z, z*x)
XX1

#susikuriame naujas reiksmes

a0 = -10
b0 = -1
a0
b0

beta = c( a0, b0, a, b)
beta

Y = XX1 %*% beta + ep
Y
yy=Y

plot(x,yy)
lines (x[z==0], yy[z==0], col=2, type = "p") 
lines(x[z==1], yy[z==1], col=3, type = "p")
abline(a,b, col=3)
abline(a0, b0, col=2)


#kompiuterio reiksmes
lm2 = lm ( yy ~ -1+z1+I(z1*x)+z+I(z*x))
lm2

lm3 = lm ( yy ~ -1+z1+x+z)
lm3

plot(x,yy)
lines (x[z==0], yy[z==0], col=2, type = "p") 
lines(x[z==1], yy[z==1], col=3, type = "p")
abline(lm3$coeff[1] ,lm3$coeff[2], col=3)
abline(lm3$coeff[3], lm3$coeff[2], col=2)

lm4 = lm ( yy ~ 1+I(z1*x)+I(z*x))
lm4
plot(x,yy)
lines (x[z==0], yy[z==0], col=2, type = "p") 
lines(x[z==1], yy[z==1], col=3, type = "p")
abline(lm4$coeff[1] ,lm4$coeff[2], col=3)
abline(lm4$coeff[1], lm4$coeff[3], col=2)
lm4$coeff



cia 2 labor



set.seed(1013)
n = 4

beta1 = runif (1, 0, 5)
beta1

beta2 = runif (1, (1/3), 3)
beta2


t = c(0 : n, n+1, n+6) #cia bus t, o t reiksme nuo 0 iki n
t #n+1 bus 5, n+6 bus 10

vienas = replicate (n+1,1) #n+1 nes turime 0
vienas

#plano matrica
XX = cbind (vienas, t)
XX

beta = c(beta1, beta2)
beta

Y=c(round(XX%*%beta, 0, NA, NA)
Y

length(Y)
length(t)

plot(t,Y)

# kai k=0
a0 <- lm (Y ~ 1)
a0
abline(a0)
points(t, predict(a0), type="b", col=2, lwd=2)

# kai k=1
a <- lm (Y ~ 1 + t)
a
plot(t, Y)
abline(a, col=3)
points(t, predict(a), type="b", col=2, lwd=2)

#kai k=2
aa <- lm (Y ~ 1 + t +I(t^2))
aa
plot(t, Y)
points(t, predict(aa), type="b", col=2, lwd=2)


# kai k=3
aaa <- lm (Y ~ 1 + t +I(t^2) + I(t^3))
aaa
plot(t, Y)
points(t, predict(aaa), type="b", col=2, lwd=2)

# kai k=4
aaaa <- lm (Y ~ 1 + t +I(t^2) + I(t^3) + I(t^4))
aaaa
plot(t, Y)
points(t, predict(aaaa), type="b", col=2, lwd=2)

# b <- function(t) {
# curve


#------------------------;
t = c(0 : n, n+1)
t

#k =0
polynomial(5, a0, 0)